In [1]:
### Loading the Dataset

try: from datasets import load_dataset
except:
    !pip install datasets
    from datasets import load_dataset


# dataset = load_dataset("nvidia/OpenMathInstruct-2", split = "train_1M")
dataset = load_dataset("commonsense_qa")
# dataset_small = dataset.select(range(10000))
# dataset_split = dataset_small.train_test_split(test_size=0.1, seed=42)
# train_data, eval_data = dataset_split["train"], dataset_split["test"]
train_data, eval_data = dataset["train"], dataset["validation"]

print(train_data)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.39k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.25M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/160k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/151k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9741 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1221 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1140 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'question', 'question_concept', 'choices', 'answerKey'],
    num_rows: 9741
})


In [2]:
eval_data.to_pandas().head()

,id,question,question_concept,choices,answerKey
0,1afa02df02c908a558b4036e80242fac,A revolving door is convenient for two directi...,revolving door,"{'label': ['A', 'B', 'C', 'D', 'E'], 'text': [...",A
1,a7ab086045575bb497933726e4e6ad28,What do people aim to do at work?,people,"{'label': ['A', 'B', 'C', 'D', 'E'], 'text': [...",A
2,b8c0a4703079cf661d7261a60a1bcbff,Where would you find magazines along side many...,magazines,"{'label': ['A', 'B', 'C', 'D', 'E'], 'text': [...",B
3,e68fb2448fd74e402aae9982aa76e527,Where are you likely to find a hamburger?,hamburger,"{'label': ['A', 'B', 'C', 'D', 'E'], 'text': [...",A
4,2435de612dd69f2012b9e40d6af4ce38,James was looking for a good place to buy farm...,farmland,"{'label': ['A', 'B', 'C', 'D', 'E'], 'text': [...",A


In [3]:
### Logging into HuggingFace

try:
  from dotenv import load_dotenv
except:
  !pip install python-dotenv
  from dotenv import load_dotenv

from huggingface_hub import login, whoami
import os

load_dotenv()
login(os.getenv("hugging_face_key2"))

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import get_peft_model, IA3Config, TaskType


model_name = "gpt2"
# model_name = "tiiuae/falcon-7b"
# model_name = "mistralai/Mistral-7B-Instruct-v0.2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [5]:
import numpy as np
def format_prompt(example):
    question = example['question']
    choices = example['choices']['text']
    labels = example['choices']['label']
    answer_key = example['answerKey']

    # Find the correct answer text
    answer_index = labels.index(answer_key)
    answer_text = choices[answer_index]

    # Create the prompt
    prompt = f"Q: {question}\nO: {' /// '.join(choices)}\nA: "
    full_text = prompt + " " + answer_text

    # Tokenize full input
    tokenized = tokenizer(
        full_text,
        padding="max_length",
        truncation=True,
        max_length=128,
    )

    # Calculate where the completion starts
    prompt_ids = tokenizer(prompt,
                           truncation=True,
                           max_length=128)["input_ids"]
    completion_start = len(prompt_ids)

    # Mask prompt tokens in the labels
    labels = [-np.inf] * completion_start + tokenized["input_ids"][completion_start:]
    labels += [-np.inf] * (128 - len(labels))  # pad label to match input

    tokenized["labels"] = labels
    return tokenized


tokenized_train = train_data.map(format_prompt, batched=False, remove_columns=train_data.column_names)
tokenized_eval = eval_data.map(format_prompt, batched=False, remove_columns=eval_data.column_names)

tokenized_train.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
tokenized_eval.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


Map:   0%|          | 0/9741 [00:00<?, ? examples/s]

Map:   0%|          | 0/1221 [00:00<?, ? examples/s]

In [6]:
tokenized_train.to_pandas().head()

,input_ids,attention_mask,labels
0,"[48, 25, 383, 9388, 1028, 262, 1524, 547, 257,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-inf, -inf, -inf, -inf, -inf, -inf, -inf, -in..."
1,"[48, 25, 44067, 2227, 284, 467, 284, 810, 262,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-inf, -inf, -inf, -inf, -inf, -inf, -inf, -in..."
2,"[48, 25, 1675, 17276, 257, 442, 11020, 407, 51...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-inf, -inf, -inf, -inf, -inf, -inf, -inf, -in..."
3,"[48, 25, 3012, 20347, 290, 584, 12763, 290, 46...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-inf, -inf, -inf, -inf, -inf, -inf, -inf, -in..."
4,"[48, 25, 383, 21831, 6807, 422, 262, 1748, 656...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-inf, -inf, -inf, -inf, -inf, -inf, -inf, -in..."


In [7]:
from transformers import pipeline
import torch
from tqdm import tqdm

# Use the PEFT model (already trained) and tokenizer
model.eval()
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0 if torch.cuda.is_available() else -1)
from transformers import pipeline
import torch
from tqdm import tqdm
from collections import defaultdict

# Use the PEFT model (already trained) and tokenizer
model.eval()
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0 if torch.cuda.is_available() else -1)

def format_prompt(example):
    question = example["question"]
    choices = example["choices"]["text"]
    prompt = f"Q: {question}\nO: {' /// '.join(choices)}\nA: "
    return prompt, choices

correct = 0
total = 0
half_correct = 0

# For concept-wise tracking
concept_stats = defaultdict(lambda: {"correct": 0.0, "total": 0})

for example in tqdm(eval_data, total=len(eval_data)):
    prompt, choices = format_prompt(example)

    # Generate prediction
    output = pipe(prompt, max_new_tokens=10, do_sample=False, eos_token_id=tokenizer.eos_token_id)[0]["generated_text"]

    # Extract generated answer (after "Answer:")
    generated = [ans.strip()  for ans in output.split("\n")[2].split("A:")[-1].strip().split(" /// ")]

    # Get ground truth
    answer_key = example["answerKey"]
    labels = example["choices"]["label"]
    true_answer = example["choices"]["text"][labels.index(answer_key)].strip()

    # Track concept-wise stats
    concept = example["question_concept"]

    if generated[0] == true_answer:
        correct += 1
        concept_stats[concept]["correct"] += 1
    elif true_answer in generated:
        correct += 0.5
        half_correct += 1
        concept_stats[concept]["correct"] += 0.5
    concept_stats[concept]["total"] += 1

    total += 1

# Final overall accuracy
accuracy = correct / total
print(f"\nOverall Accuracy: {accuracy:.2%}")
print(f"Half-correct answers: {half_correct} out of {total}")

# # Per-concept accuracy
# print("\nPer-concept Accuracy:")
# for concept, stats in sorted(concept_stats.items(), key=lambda x: x[1]["total"], reverse=True):
#     acc = stats["correct"] / stats["total"]
#     print(f"{concept:<20}: {acc:.2%} ({stats['total']} examples)")

# def format_prompt(example):
#     question = example["question"]
#     choices = example["choices"]["text"]
#     prompt = f"Q: {question}\nO: {' /// '.join(choices)}\nA: "
#     return prompt, choices

# correct = 0
# total = 0
# half_correct = 0

# for example in tqdm(eval_data, total=len(eval_data)):
#     prompt, choices = format_prompt(example)

#     # Generate prediction
#     output = pipe(prompt, max_new_tokens=10, do_sample=False, eos_token_id=tokenizer.eos_token_id)[0]["generated_text"]

#     # Extract generated answer (after "Answer:")
#     generated = [ans.strip()  for ans in output.split("\n")[2].split("A:")[-1].strip().split(" /// ")]


#     # Get ground truth
#     answer_key = example["answerKey"]
#     labels = example["choices"]["label"]
#     true_answer = example["choices"]["text"][labels.index(answer_key)].strip()

#     # print("\n".join(output.split("\n")[:3]))
#     # print(f"\n\nModel: {generated[0]} and Actual Answer: {true_answer}")

#     if generated[0] == true_answer:
#         correct += 1
#     elif true_answer in generated:
#         correct += 0.5
#         half_correct += 1
#     total += 1

# accuracy = correct / total
# print(f"Accuracy: {accuracy:.2%}")


Device set to use cuda:0
Device set to use cuda:0
100%|██████████| 1221/1221 [02:09<00:00,  9.40it/s]


Overall Accuracy: 0.04%
Half-correct answers: 1 out of 1221


In [8]:
import pandas as pd
csv_url = "https://raw.githubusercontent.com/MSDS-Capstone-Project/LLM_FineTuning/refs/heads/main/eval_data_concept.csv"
concept_map_df = pd.read_csv(csv_url)


concept_df = pd.DataFrame([
    {"question_concept": concept, "accuracy": stats["correct"] / stats["total"], "total": stats["total"]}
    for concept, stats in concept_stats.items()
])


merged_df = concept_df.merge(concept_map_df, on="question_concept", how="left")


major_agg = (
    merged_df
    .groupby("Major Concepts")
    .apply(lambda g: pd.Series({
        "accuracy": (g["accuracy"] * g["total"]).sum() / g["total"].sum(),
        "questions": g["total"].sum()
    }))
    .reset_index()
    .sort_values("accuracy", ascending=False)
    .reset_index(drop=True)
)

styled_df = (
    major_agg
    .style
    .background_gradient(subset=["accuracy"], cmap="Greens")
    .format({"accuracy": "{:.2%}", "questions": "{:,.0f}"})
    .set_caption("PEFT Accuracy by Major Concept")
)

styled_df


<ipython-input-8-c5cc8de28878>:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({


,Major Concepts,accuracy,questions
0,Buildings & Spaces,0.57%,88
1,Abstract Concepts & Events,0.00%,122
2,Emotions & Mental States,0.00%,66
3,Food & Drink,0.00%,76
4,Human Activities & Daily Life,0.00%,166
5,Nature & Environment,0.00%,206
6,Objects & Tools,0.00%,187
7,Society & Culture,0.00%,168
8,Sports & Physical Activities,0.00%,65
9,Technology & Media,0.00%,77


In [9]:
model.eval()
import torch
# prompt = f"Q: {question}\nChoices:\n"
#     for choice in choices:
#         prompt += f"- {choice}\n"
#     prompt += "Answer:"
q = "What do people aim to do at work?"
options = ['complete job', 'learn from each other', 'kill animals', 'wear hats', 'talk to each other']
options2 = ' /// '.join(options)
from IPython.display import display, Math
prompt = f"Q: {q}\nO: {options2}\nA: "
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    outputs = model.generate(
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    max_new_tokens=10,
    do_sample=False,
    pad_token_id=tokenizer.eos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    early_stopping=True,
    num_beams=5,  # Optional: beam search for stronger outputs
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=False))

Q: What do people aim to do at work?
O: complete job /// learn from each other /// kill animals /// wear hats /// talk to each other
A: _______________________________________________________________________________
A: _______________________________________________________________________________


In [14]:
import torch
import math
from tqdm import tqdm
from transformers import DataCollatorForLanguageModeling

# Make sure your model is in evaluation mode
model.eval()

# Step 1: Convert eval_data to plain text (based on your QA-style format)
eval_texts = [
    f"Q: {ex['question']} O: {' /// '.join(ex['choices']['text'])} A:"
    for ex in eval_data
]

# Step 2: Tokenize
tokenized = tokenizer(
    eval_texts,
    return_tensors="pt",
    padding=True,
    truncation=True,
    max_length=512
)

# Step 3: Prepare DataLoader
dataset = torch.utils.data.TensorDataset(tokenized['input_ids'], tokenized['attention_mask'])
dataloader = torch.utils.data.DataLoader(dataset, batch_size=8)

# Step 4: Compute total loss and tokens
total_loss = 0
total_tokens = 0

device = model.device  # Ensure model and tensors are on same device

with torch.no_grad():
    for batch in tqdm(dataloader):
        input_ids, attention_mask = [b.to(device) for b in batch]

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=input_ids)
        loss = outputs.loss
        num_tokens = attention_mask.sum().item()

        total_loss += loss.item() * num_tokens
        total_tokens += num_tokens

# Step 5: Final Perplexity
average_loss = total_loss / total_tokens
perplexity = math.exp(average_loss)

print(f"Perplexity: {perplexity:.2f}")


100%|██████████| 153/153 [00:09<00:00, 16.50it/s]

Perplexity: 4449.39
